<div style="text-align:center">
    <h1>
        VSL RL : SUMO Simulation (DQL)
    </h1>
</div>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam
import sumo_env as env
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
import gym
import datetime

# Load the TensorBoard notebook extension
from tensorflow.keras.callbacks import TensorBoard
%load_ext tensorboard


/Users/asmae/miniforge3/envs/dql_vsl/lib/python3.8/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Initialize the environment

In [2]:
import gym
from gym.envs.registration import register
from SUMOInitializeEnv import SUMOEnv_Initializer
register(
    id='SumoGUI-v0',
    entry_point='SUMOInitializeEnv:SUMOEnv_Initializer'
)
env = gym.make('SumoGUI-v0')
np.random.seed(0)
env.seed(0)
tf.__version__

'2.8.0'

Build model

In [3]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(env.action_space.size))
model.add(Activation('linear'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 2)                 0         
                                                                 
 dense (Dense)               (None, 16)                48        
                                                                 
 activation (Activation)     (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 102       
                                                                 
 activation_1 (Activation)   (None, 6)                 0         
                                                                 
Total params: 150
Trainable params: 150
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
%reload_ext tensorboard
log_dir = "/users/asmae/Desktop/PHD/rl_vsl_sumo/rl_vsl_sumo/gym_sumo/envs/logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = [TensorBoard(log_dir=log_dir, histogram_freq=1,write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)]

SyntaxError: invalid syntax (1844678023.py, line 3)

#### Deep Q Learning

In [9]:
import epsilon_greedy as EpsilonGreedy
#tf.debugging.experimental.enable_dump_debug_info("/users/asmae/Desktop/PHD/TS_LOGDIR", tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=50000, window_length=1)
dqn = DQNAgent(model=model, nb_actions=env.action_space.size, memory=memory, nb_steps_warmup=10,
target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="/users/asmae/Desktop/PHD/TS_LOGDIR", histogram_freq=1)
# Okay, now it's time to learn something! We visualize the training here for show, but this slows down training quite a lot. 
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
dqn.fit(env, nb_steps=2000, visualize=True, verbose=2, 
          callbacks=tensorboard_callback)
env.close()
dqn.test(env, nb_episodes=1, nb_max_episode_steps= 2000, visualize=True)
env.close()

2022-03-05 12:45:12.743019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-05 12:45:12.752809: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-05 12:45:12.775808: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


AttributeError: 'TensorBoard' object has no attribute '_should_trace'

In [11]:
%tensorboard --logdir={log_folder}